#                        Text Extraction from Raw data using NER (Name Entity Recognition)
![title](pics/ner.png)
![title](pics/nercsv.png)

In [ ]:
import re
from itertools import combinations
from os import path
import pandas as pd
import csv

year = "data/HEPabstracts/2003"
filename = 301001
filesize = 304271

count = 0
flag = 0
paper = ""
date = ""
title = ""
author = ""
journal = ""
abstract = ""

columns = ["Paper", "Date", "Title", "Authors", "Journal-ref", "Abstract"]
#df = pd.read_csv("abstracts.csv", sep=",", index_col=False)

#Paper;Date;Title;Authors;Journal-ref;Abstract

while filename <= filesize:
    if(path.exists(year + '/0'+ str(filename) +'.abs')):
        with open (year + '/0'+ str(filename) +'.abs', 'rt') as myfile:
            for myline in myfile:
                if(myline.find("Paper") == 0):
                    paper = myline.strip()
                    paper = paper[paper.find(":")+9:]
                    if(paper[0] == '9'):
                        paper = "19" + paper
                    else:
                        paper = "20" + paper
                    
                elif(myline.find("Date") == 0):
                    date = myline.strip()
                    date = date[date.find(":")+2:]
                    
                elif(myline.find("Title") == 0):
                    title = myline.strip()
                    title = title[title.find(":")+2:]
                    
                elif(myline.find("Author") == 0):
                    author = myline.strip()
                    author = author[author.find(":")+2:]
                    if(author.find("(") > 0):
                        author = author[:author.find("(")-1]
                    author = re.split(', and |, | and ',author)
                    temp = ""
                    for a in author:
                        temp = temp + a
                        temp = temp + ", "
                    author = temp
                    
                elif((myline.find("Journal-ref") == 0) or (myline.find("Report-no") == 0)):
                    journal = myline.strip()
                    journal = journal[journal.find(":")+2:]
                
                elif(((myline.find("\\") == 0) and (count > 2)) or flag==1):
                    flag = 1
                    if(myline.find("\\") != 0):
                        temp = myline.strip()
                        abstract = abstract + temp
                        abstract = abstract + " "
                        
                count = count + 1
                
            pa = pd.DataFrame()
            df = pa.append({'Paper':paper, 'Date':date, 'Title':title, 'Authors':author, 'Journal-ref':journal, 'Abstract':abstract}, ignore_index=True)
            df.to_csv('data/abstracts.csv', mode='a', header=False, index=False, sep=',', encoding='utf-8', line_terminator="\n", columns=columns)
            
            count = 0
            flag = 0
            paper = ""
            date = ""
            title = ""
            author = ""
            journal = ""
            abstract = ""
    filename = filename + 1
    




##                                Author Citation Network

In [ ]:
import re
from itertools import combinations
from os import path
import pandas as pd
import csv

df = pd.read_csv("data/abstracts.csv", sep=",", index_col=False)
authorCitation = open("data/authorCitation.txt","a")

if(path.exists('data/Cit-HepTh.txt')):
    with open ('data/Cit-HepTh.txt', 'rt') as myfile:
        for myline in myfile:
            myline = myline.strip()
            myline = re.split('\t',myline)
            paper1 = myline[0]
            paper2 = myline[1]
            if(len(paper1) == 4):
                paper1 = "20000" + paper1
            elif(len(paper1) == 5):
                paper1 = "2000" + paper1
            elif(len(paper1) == 6):
                paper1 = "200" + paper1
            elif(len(paper1) == 7):
                paper1 = "19" + paper1

            if(len(paper2) == 4):
                paper2 = "20000" + paper2
            elif(len(paper2) == 5):
                paper2 = "2000" + paper2
            elif(len(paper2) == 6):
                paper2 = "200" + paper2
            elif(len(paper2) == 7):
                paper2 = "19" + paper2

            a = df.loc[df['Paper'] == int(paper1)]
            author1 =  a.iat[0,3]
            author1 = author1.strip()
            author1 = author1[:-1]
            if(author1.find(',') > -1):
                author1 = re.split(', ',author1)

            b = df.loc[df['Paper'] == int(paper2)]
            author2 = b.iat[0,3]
            author2 = author2.strip()
            author2 = author2[:-1]
            if(author2.find(',') > -1):
                author2 = re.split(', ',author2)

            if((type(author1) is str) and (type(author2) is str)):
                authorCitation.write(author1 + "\t" + author2)
                authorCitation.write("\n")
            elif((type(author1) is str) and (type(author2) is list)):
                for x in author2:
                    authorCitation.write(author1 + "\t" + x)
                    authorCitation.write("\n")
            elif((type(author1) is list) and (type(author2) is str)):
                for x in author1:
                    authorCitation.write(x + "\t" + author2)
                    authorCitation.write("\n")
            elif((type(author1) is list) and (type(author2) is list)):
                for x in author1:
                    for y in author2:
                        authorCitation.write(x + "\t" + y)
                        authorCitation.write("\n")



The input of this code would be the abstract csv file that we made earlier.
Output of this file would be the citations of authors that which author is citing which author.
The Left column is indicating the author which is citing and the right column indicates the author that are being cited.

##                                        Author Colloboration Network

In [ ]:
import re
from itertools import combinations
from os import path
year = "2003"
filename = 301001
filesize = 304271
authorFile = open("data/AuthorCollaboration.txt","a")
while filename <= filesize:
    if(path.exists(year + '/0'+ str(filename) +'.abs')):
        with open (year + '/0'+ str(filename) +'.abs', 'rt') as myfile:
            for myline in myfile:
                if(myline.find("Author") == 0):
                    myline = myline.strip()
                    myline = myline[9:]
                    if(myline.find("(") > 0):
                        myline = myline[:myline.find("(")-1]
                    myline = re.split(', and |, | and ',myline)

                    if(len(myline) > 1):
                        comb = combinations(myline, 2)
                        for i in list(comb): 
                            authorFile.write(i[0] + "\t" + i[1])
                            authorFile.write("\n")
    filename = filename + 1




The Output file shows which authors collaboratively worked together
he Output indicates that the two author have worked together on one or more documents .
